# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

## 1. Setting the environment

In [1]:
# Clone the repository
!git clone https://github.com/karpathy/nanochat.git
%cd nanochat

Cloning into 'nanochat'...
remote: Enumerating objects: 928, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 928 (delta 70), reused 30 (delta 30), pack-reused 812 (from 2)
Receiving objects: 100% (928/928), 1.14 MiB | 20.49 MiB/s, done.
Resolving deltas: 100% (562/562), done.
/root/nanochat


In [2]:
# Set the base directory for artifacts
import os
os.environ['NANOCHAT_BASE_DIR'] = '/content/nanochat_data'
!mkdir -p $NANOCHAT_BASE_DIR

# Install uv package manager
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv and cargo to the PATH
os.environ['PATH'] = f"/root/.cargo/bin:/root/.local/bin:{os.environ['PATH']}"

# Create a virtual environment and install dependencies
!uv venv
!uv sync
print('-'*40)
print('Environment Set')

# Install Rust/Cargo for the tokenizer
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
print('-'*40)
print('Installed Rust/Cargo')

downloading uv 0.9.24 x86_64-unknown-linux-gnu
no checksums to verify
installing to /root/.local/bin
  uv
  uvx
everything's installed!

To add $HOME/.local/bin to your PATH, either restart your shell or run:

    source $HOME/.local/bin/env (sh, bash, zsh)
    source $HOME/.local/bin/env.fish (fish)
WARN: The following commands are shadowed by other commands in your PATH: uv uvx
Using CPython 3.10.19
Creating virtual environment at: .venv
Resolved 134 packages in 31ms
⠙ Preparing packages... (0/119)
⠙ Preparing packages... (0/119)
⠙ Preparing packages... (0/119)
⠙ Preparing packages... (0/119)
pytz                 ------------------------------     0 B/497.29 KiB
⠙ Preparing packages... (0/119)
pytz                 ------------------------------ 14.91 KiB/497.29 KiB
⠙ Preparing packages... (0/119)
pytz                 ------------------------------ 14.91 KiB/497.29 KiB
⠙ Preparing packages... (0/119)
pytz                 ------------------------------ 14.91 KiB/497.29 KiB
⠙ Preparing 

In [26]:
!bash -lc "cargo install maturin"

    Updating crates.io index
  Downloaded maturin v1.11.5
  Downloaded 1 crate (215.5KiB) in 0.06s
  Installing maturin v1.11.5
    Updating crates.io index
     Locking 345 packages to latest compatible versions
      Adding cargo-xwin v0.18.6 (available: v0.20.2)
      Adding cargo_metadata v0.20.0 (available: v0.23.1)
      Adding dialoguer v0.11.0 (available: v0.12.0)
      Adding dirs v5.0.1 (available: v6.0.0)
      Adding generic-array v0.14.7 (available: v0.14.9)
      Adding toml_edit v0.23.10+spec-1.0.0 (available: v0.24.0+spec-1.1.0)
      Adding ureq v2.12.1 (available: v3.1.4)
      Adding zip v6.0.0 (available: v7.0.0)
  Downloaded anstyle v1.0.13
  Downloaded globset v0.4.18
  Downloaded mime v0.3.17
  Downloaded subtle v2.6.1
  Downloaded toml_datetime v0.6.11
  Downloaded untrusted v0.9.0
  Downloaded urlencoding v2.1.3
  Downloaded yoke-derive v0.8.1
  Downloaded zstd-safe v7.2.4
  Downloaded zstd v0.13.3
  Downloaded zopfli v0.8.3
  Downloaded tracing-subscriber v0.3

In [27]:
!bash -lc "which maturin && maturin --version"

/root/.cargo/bin/maturin
maturin 1.11.5


In [3]:
# Reset the report directory
!bash -c "source .venv/bin/activate && python -m nanochat.report reset"

# Download the eval_bundle for CORE metric evaluation
!if [ ! -d "$NANOCHAT_BASE_DIR/eval_bundle" ]; then \
    curl -L -o eval_bundle.zip https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip && \
    unzip -q eval_bundle.zip && \
    rm eval_bundle.zip && \
    mv eval_bundle $NANOCHAT_BASE_DIR; \
fi

# Download the pre-training data shards
# For the d34 model, 240 shards are recommended (~24GB)
!bash -c "source .venv/bin/activate && python -m nanochat.dataset -n 240"

print('Data download and report reset complete')

Reset report and wrote header to /content/nanochat_data/report/header.md
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.8M  100 24.8M    0     0  16.4M      0  0:00:01  0:00:01 --:--:-- 16.4M
Target directory: /content/nanochat_data/base_data

Successfully downloaded shard_00000.parquet
Successfully downloaded shard_00045.parquet
Successfully downloaded shard_00015.parquet
Successfully downloaded shard_00030.parquet
Successfully downloaded shard_00001.parquet
Successfully downloaded shard_00002.parquet
Successfully downloaded shard_00031.parquet
Successfully downloaded shard_00016.parquet
Successfully downloaded shard_00046.parquet
Successfully downloaded shard_00003.parquet
Successfully downloaded shard_00032.parquet
Successfully downloaded shard_00017.parquet
Successfully downloaded shard_00047.parquet
Successfully downloaded shard_00004.parquet
Successfully downloade

## 2. Training the Tokenizer

In [28]:
# Building the Tokenizer
!bash -c "source .venv/bin/activate && uv run maturin develop --release --manifest-path rustbpe/Cargo.toml"

💥 maturin failed
  Caused by: manifest path `rustbpe/Cargo.toml` does not exist or is invalid
  Caused by: No such file or directory (os error 2)


In [ ]:
# Train the tokenizer. We skip this but you can train it if you want to experiment
#!bash -c "source .venv/bin/activate && python -m scripts.tok_train --max_chars=2000000000"

## 3. Downloading the Tokenizer from Huggingface Repositoriy

In [29]:
from huggingface_hub import hf_hub_download
# Download the specific file and get its local file path
tokenizer_pkl_path = hf_hub_download(repo_id="karpathy/nanochat-d34", filename="tokenizer.pkl")
tokenizer_pt_path = hf_hub_download(repo_id="karpathy/nanochat-d34", filename="token_bytes.pt")

In [31]:
tokenizer_pkl_path

'/root/.cache/huggingface/hub/models--karpathy--nanochat-d34/snapshots/c48357d43863a3a6cdc5f5db5b4ec5964e4192d6/tokenizer.pkl'

In [32]:
# Copying the file into the correct folder
!mkdir '../../content/nanochat_data/tokenizer/'
!cp '/root/.cache/huggingface/hub/models--karpathy--nanochat-d34/snapshots/c48357d43863a3a6cdc5f5db5b4ec5964e4192d6/tokenizer.pkl' '/content/nanochat_data/tokenizer/tokenizer.pkl'
!cp '/root/.cache/huggingface/hub/models--karpathy--nanochat-d34/snapshots/c48357d43863a3a6cdc5f5db5b4ec5964e4192d6/token_bytes.pt' '/content/nanochat_data/tokenizer/token_bytes.pt'

In [35]:
# Evaluate the tokenizer
!bash -c "source .venv/bin/activate && python -m scripts.tok_eval"

print('Tokenizer training and evaluation complete.')


Vocab sizes:
GPT-2: 50257
GPT-4: 100277
Ours: 65536

Comparison with GPT-2:
Text Type  Bytes    GPT-2           Ours            Relative     Better    
                    Tokens  Ratio   Tokens  Ratio   Diff %      
-----------------------------------------------------------------------------------------------
news       1819     404     4.50    372     4.89       +7.9%     Ours      
korean     893      745     1.20    710     1.26       +4.7%     Ours      
code       1259     576     2.19    492     2.56      +14.6%     Ours      
math       1834     936     1.96    966     1.90       -3.2%     GPT-2     
science    1112     260     4.28    225     4.94      +13.5%     Ours      
fwe-train  4208518  900364  4.67    856992  4.91       +4.8%     Ours      
fwe-val    4908443  1059062 4.63    1010217 4.86       +4.6%     Ours      

Comparison with GPT-4:
Text Type  Bytes    GPT-4           Ours            Relative     Better    
                    Tokens  Ratio   Tokens  Ratio   Di

## 4. Base Model Pre-training 

You can train the 561M parameter d20 model from scratch. We'll skip this part because Andrej Karparthy already trained it for 4 days with a $2,500 budget

We set --device_batch_size=4. If you encounter an OOM error, reduce this to 2 or 1.
We set --depth=4 for a quicker run to test the pipeline. Default is --depth=20. It will take estimated ~12-20 hours to execute the complete pipeline using a A100-80GB GPU.

In [27]:
#!bash -c "source .venv/bin/activate && python -m scripts.base_train --depth=4 --device_batch_size=4"

In [36]:
# Download the specific file and get its local file path
d34_gpt_model_path = hf_hub_download(repo_id="karpathy/nanochat-d34", filename="model_169150.pt")
d34_gpt_json_path = hf_hub_download(repo_id="karpathy/nanochat-d34", filename="meta_169150.json")

model_169150.pt:   0%|          | 0.00/8.58G [00:00<?, ?B/s]

meta_169150.json: 0.00B [00:00, ?B/s]

In [40]:
# Move the correct weights to the correct location of our environment to run the program
!mkdir '../../content/nanochat_data/base_checkpoints/'
!mkdir '../../content/nanochat_data/base_checkpoints/d34'
!cp '/root/.cache/huggingface/hub/models--karpathy--nanochat-d34/snapshots/c48357d43863a3a6cdc5f5db5b4ec5964e4192d6/model_169150.pt' '/content/nanochat_data/base_checkpoints/d34/model_169150.pt'
!cp '/root/.cache/huggingface/hub/models--karpathy--nanochat-d34/snapshots/c48357d43863a3a6cdc5f5db5b4ec5964e4192d6/meta_169150.json' '/content/nanochat_data/base_checkpoints/d34/meta_169150.json'

## 5. Evaluation of the Pretrained Model

In [36]:
# Evaluate the model on a larger chunk of train/val data and draw some samples
!bash -c "source .venv/bin/activate && python -m scripts.base_loss"

# Evaluate the model on CORE tasks
!bash -c "source .venv/bin/activate && python -m scripts.base_eval"

Autodetected device type: cuda
2026-01-11 04:23:44,719 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 04:23:44,720 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 04:23:44,720 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/base_checkpoints/d34 with step 169150
2026-01-11 04:23:50,361 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}
train bpb: 0.6995
val bpb: 0.6984
<|bos|>The capital of France is Paris. It is the most populous city in France and the center of the metropolitan
<|bos|>The chemical symbol of gold is Au. It is a soft, yellow metal that is one of the most malleable
<|bos|>If yesterday was Friday, then tomorrow will be Saturday. If today is Monday, then tomorrow will be Tuesday. If today is
<|bos|>The opposite of hot is cold. The opposite of hot is cold. 

## 6. Midtraining the Model

This part of the training is more related to make the model understand conversational structure, special tokens, and tool use.

In [44]:
# device_batch_size is always <= batch size used in training step
!bash -c "source .venv/bin/activate && python -m scripts.mid_train --device_batch_size=4" 

Autodetected device type: cuda
2026-01-11 14:19:00,840 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 14:19:00,863 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 14:19:00,863 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/base_checkpoints/d34 with step 169150
2026-01-11 14:19:07,587 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}
Tokens / micro-batch / rank: 4 x 2048 = 8,192
Tokens / micro-batch: 8,192
Total batch size 524,288 => gradient accumulation steps: 64
Scaling the LR for the AdamW parameters ∝1/√(2176/768) = 0.594089
--------------------------------------------------------------------------------
HINT (Oct 21 2025)
If you recently did a git pull and suddely see this, it might be due to the new addition of identity conversations
See this discussion for mor

In [61]:
!bash -c "source .venv/bin/activate && python -m scripts.chat_eval -i mid"

Autodetected device type: cuda
2026-01-11 19:33:29,543 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 19:33:29,566 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 19:33:29,567 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/mid_checkpoints/d34 with step 811
2026-01-11 19:33:38,624 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}
Final: 1688/2376 (71.04%)
ARC-Easy accuracy: 71.04%
Final: 636/1172 (54.27%)
ARC-Challenge accuracy: 54.27%
Final: 6048/14042 (43.07%)
MMLU accuracy: 43.07%
Rank 0 | 151/1319 (11.45%)
Final: 151/1319 (11.45%)
GSM8K accuracy: 11.45%
README.md: 6.52kB [00:00, 17.5MB/s]
openai_humaneval/test-00000-of-00001.par(…): 100%|█| 83.9k/83.9k [00:02<00:00, 3
Generating test split: 100%|████████| 164/164 [00:00<00:00, 24477.47 examples/s]
Rank 0 | 16/164 

## 8. Supervised Fine-Tuning (SFT)

Finally, we perform supervised fine-tuning for domain adaptation, making the model a better chatbot.

In [60]:
# The device_batch_size must be <= the batch size used in base training
!bash -c "source .venv/bin/activate && python -m scripts.chat_sft --device_batch_size=4"

Autodetected device type: cuda
2026-01-11 18:52:11,367 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 18:52:11,400 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 18:52:11,401 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/mid_checkpoints/d34 with step 811
2026-01-11 18:52:17,570 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}
README.md: 9.00kB [00:00, 24.7MB/s]
ARC-Easy/train-00000-of-00001.parquet: 100%|██| 331k/331k [00:02<00:00, 151kB/s]
ARC-Easy/test-00000-of-00001.parquet: 100%|███| 346k/346k [00:01<00:00, 182kB/s]
ARC-Easy/validation-00000-of-00001.parqu(…): 100%|█| 86.1k/86.1k [00:01<00:00, 5
Generating validation split: 100%|█| 570/570 [00:00<00:00, 191505.39 examples/s]
ARC-Challenge/train-00000-of-00001.parqu(…): 100%|█| 190k/190k [00:02<00:00, 91.
ARC-C

## 9. Evaluate the SFT Model

In [103]:
!bash -c "source .venv/bin/activate && python -m scripts.chat_eval -i sft"

Autodetected device type: cuda
2026-01-11 21:53:33,844 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 21:53:33,867 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 21:53:33,867 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/chatsft_checkpoints/d34 with step 669
2026-01-11 21:53:40,849 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}
Final: 1724/2376 (72.56%)
ARC-Easy accuracy: 72.56%
Final: 632/1172 (53.92%)
ARC-Challenge accuracy: 53.92%
Final: 6134/14042 (43.68%)
MMLU accuracy: 43.68%
Rank 0 | 162/1319 (12.28%)
Final: 162/1319 (12.28%)
GSM8K accuracy: 12.28%
Rank 0 | 12/164 (7.32%)
Final: 12/164 (7.32%)
HumanEval accuracy: 7.32%
Rank 0 | 254/256 (99.22%)
Final: 254/256 (99.22%)
SpellingBee accuracy: 99.22%


## 10. Downloading SFT Trained Weights

Because you need almost more than 2 hours for Midtraining and Supervised Fine Tuning

In [104]:
# Download the specific file and get its local file path
d34_stf_gpt_model_path = hf_hub_download(repo_id="renatocastro33/nanochat-d34-sft", filename="chatsft_checkpoints/d34/model_000669.pt")
d34_sft_gpt_json_path = hf_hub_download(repo_id="renatocastro33/nanochat-d34-sft", filename="chatsft_checkpoints/d34/meta_000669.json")

chatsft_checkpoints/d34/model_000669.pt:   0%|          | 0.00/8.58G [00:00<?, ?B/s]

meta_000669.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [105]:
d34_stf_gpt_model_path

'/root/.cache/huggingface/hub/models--renatocastro33--nanochat-d34-sft/snapshots/8e0266180c5c395b6f53e6625347370e2db645e7/chatsft_checkpoints/d34/model_000669.pt'

In [48]:
!mkdir '../../content/nanochat_data/chatsft_checkpoints/'

In [49]:
!mkdir '../../content/nanochat_data/chatsft_checkpoints/d34'

In [107]:
!cp '/root/.cache/huggingface/hub/models--renatocastro33--nanochat-d34-sft/snapshots/8e0266180c5c395b6f53e6625347370e2db645e7/chatsft_checkpoints/d34/model_000669.pt' '/content/nanochat_data/chatsft_checkpoints/d34/model_000669.pt'
!cp '/root/.cache/huggingface/hub/models--renatocastro33--nanochat-d34-sft/snapshots/8e0266180c5c395b6f53e6625347370e2db645e7/chatsft_checkpoints/d34/meta_000669.json' '/content/nanochat_data/chatsft_checkpoints/d34/meta_000669.json'

In [109]:
!bash -c "source .venv/bin/activate && python -m nanochat.report generate"

# Display the final report
from IPython.display import display, Markdown
with open('report.md', 'r') as f:
    report_content = f.read()
display(Markdown(report_content))

Generating report to /content/nanochat_data/report/report.md
Copying report.md to current directory for convenience


# nanochat training report

Generated: 2026-01-11 13:45:33

## Environment

### Git Information
- Branch: master
- Commit: f5a0ea4 (clean)
- Message: take out these gitignore dirs

### Hardware
- Platform: Linux
- CPUs: 93 cores (93 logical)
- Memory: 1842.1 GB
- GPUs: 1x NVIDIA H100 80GB HBM3
- GPU Memory: 79.2 GB total
- CUDA Version: 12.8
- Hourly Rate: $3.00/hour

### Software
- Python: 3.10.19
- PyTorch: 2.9.0+cu128


### Bloat
- Characters: 379,889
- Lines: 8,847
- Files: 46
- Tokens (approx): 94,972
- Dependencies (uv.lock lines): 3,604

Run started: 2026-01-11 13:45:34

---

## Tokenizer evaluation
timestamp: 2026-01-11 14:04:30

### Comparison with GPT-2

| Text Type | Bytes | GPT-2 Tokens | GPT-2 Ratio | Ours Tokens | Ours Ratio | Relative Diff % |
|-----------|-------|--------------|--------------|-------------|------------|-----------------|
| news | 1819 | 404 | 4.50 | 372 | 4.89 | +7.9% |
| korean | 893 | 745 | 1.20 | 710 | 1.26 | +4.7% |
| code | 1259 | 576 | 2.19 | 492 | 2.56 | +14.6% |
| math | 1834 | 936 | 1.96 | 966 | 1.90 | -3.2% |
| science | 1112 | 260 | 4.28 | 225 | 4.94 | +13.5% |
| fwe-train | 4208518 | 900364 | 4.67 | 856992 | 4.91 | +4.8% |
| fwe-val | 4908443 | 1059062 | 4.63 | 1010217 | 4.86 | +4.6% |

### Comparison with GPT-4

| Text Type | Bytes | GPT-4 Tokens | GPT-4 Ratio | Ours Tokens | Ours Ratio | Relative Diff % |
|-----------|-------|--------------|--------------|-------------|------------|-----------------|
| news | 1819 | 387 | 4.70 | 372 | 4.89 | +3.9% |
| korean | 893 | 364 | 2.45 | 710 | 1.26 | -95.1% |
| code | 1259 | 309 | 4.07 | 492 | 2.56 | -59.2% |
| math | 1834 | 832 | 2.20 | 966 | 1.90 | -16.1% |
| science | 1112 | 249 | 4.47 | 225 | 4.94 | +9.6% |
| fwe-train | 4208518 | 874799 | 4.81 | 856992 | 4.91 | +2.0% |
| fwe-val | 4908443 | 1029691 | 4.77 | 1010217 | 4.86 | +1.9% |


## Midtraining
timestamp: 2026-01-11 17:59:54

- run: dummy
- device_type: 
- dtype: bfloat16
- model_tag: None
- model_step: None
- num_iterations: -1
- max_seq_len: 2048
- device_batch_size: 4
- total_batch_size: 524,288
- embedding_lr: 0.2000
- unembedding_lr: 0.0040
- matrix_lr: 0.0200
- weight_decay: 0.0000
- init_lr_frac: 1.0000
- eval_every: 150
- eval_tokens: 10,485,760
- dry_run: False
- Number of iterations: 811
- DDP world size: 1
- Minimum validation bpb: 0.3282


## Chat evaluation mid
timestamp: 2026-01-11 20:42:40

- source: mid
- task_name: None
- dtype: bfloat16
- temperature: 0.0000
- max_new_tokens: 512
- num_samples: 1
- top_k: 50
- batch_size: 8
- model_tag: None
- step: None
- max_problems: None
- device_type: 
- ARC-Easy: 0.7104
- ARC-Challenge: 0.5427
- MMLU: 0.4307
- GSM8K: 0.1145
- HumanEval: 0.0976
- SpellingBee: 0.9922
- ChatCORE metric: 0.4082


## Chat SFT
timestamp: 2026-01-11 19:11:55

- run: dummy
- device_type: 
- dtype: bfloat16
- source: mid
- model_tag: None
- model_step: None
- num_epochs: 1
- num_iterations: -1
- device_batch_size: 4
- target_examples_per_step: 32
- embedding_lr: 0.2000
- unembedding_lr: 0.0040
- matrix_lr: 0.0200
- weight_decay: 0.0000
- init_lr_frac: 0.0200
- eval_every: 100
- eval_steps: 100
- eval_metrics_every: 200
- eval_metrics_max_problems: 1024
- Training rows: 21,443
- Number of iterations: 670
- Training loss: 0.5284
- Validation loss: 0.8004


## Chat evaluation sft
timestamp: 2026-01-11 23:01:23

- source: sft
- task_name: None
- dtype: bfloat16
- temperature: 0.0000
- max_new_tokens: 512
- num_samples: 1
- top_k: 50
- batch_size: 8
- model_tag: None
- step: None
- max_problems: None
- device_type: 
- ARC-Easy: 0.7256
- ARC-Challenge: 0.5392
- MMLU: 0.4368
- GSM8K: 0.1228
- HumanEval: 0.0732
- SpellingBee: 0.9922
- ChatCORE metric: 0.4095


## Summary

- Characters: 379,889
- Lines: 8,847
- Files: 46
- Tokens (approx): 94,972
- Dependencies (uv.lock lines): 3,604

| Metric          | BASE     | MID      | SFT      | RL       |
|-----------------|----------|----------|----------|----------|
| ARC-Challenge   | -        | 0.5427   | 0.5392   | -        |
| ARC-Easy        | -        | 0.7104   | 0.7256   | -        |
| GSM8K           | -        | 0.1145   | 0.1228   | -        |
| HumanEval       | -        | 0.0976   | 0.0732   | -        |
| MMLU            | -        | 0.4307   | 0.4368   | -        |
| ChatCORE        | -        | 0.4082   | 0.4095   | -        |

Total wall clock time: 9h15m


## 11. Inference

In [108]:
# Chat with the model via the Command Line Interface (CLI)
!bash -c 'source .venv/bin/activate && python -m scripts.chat_cli -p "Why is the sky blue?"'

Autodetected device type: cuda
2026-01-11 23:07:40,074 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 23:07:40,137 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 23:07:40,138 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/chatsft_checkpoints/d34 with step 669
2026-01-11 23:07:46,736 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}

NanoChat Interactive Mode
--------------------------------------------------
Type 'quit' or 'exit' to end the conversation
Type 'clear' to start a new conversation
--------------------------------------------------

Assistant: The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the short

In [113]:
# Chat with the model via the Command Line Interface (CLI)
!bash -c 'source .venv/bin/activate && python -m scripts.chat_cli -p "If x = 3, how is 3*x + 5?"'

Autodetected device type: cuda
2026-01-11 23:21:14,179 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 23:21:14,217 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 23:21:14,217 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/chatsft_checkpoints/d34 with step 669
2026-01-11 23:21:20,660 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}

NanoChat Interactive Mode
--------------------------------------------------
Type 'quit' or 'exit' to end the conversation
Type 'clear' to start a new conversation
--------------------------------------------------

Assistant: To evaluate the expression x*3 + 5, we can simply multiply the numbers together. 

3*3 equals 9, and then adding 5 results in 14. Therefore, the final expression is 3*3 + 5, which simplifies to 9 + 5, giving us 14.

In [114]:
# Chat with the model via the Command Line Interface (CLI)
!bash -c 'source .venv/bin/activate && python -m scripts.chat_cli -p "Calculate 5*x + 3 = 13, then x is equal to ?"'

Autodetected device type: cuda
2026-01-11 23:22:30,079 - nanochat.common - INFO - Distributed world size: 1
2026-01-11 23:22:30,147 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 23:22:30,147 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/chatsft_checkpoints/d34 with step 669
2026-01-11 23:22:36,687 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}

NanoChat Interactive Mode
--------------------------------------------------
Type 'quit' or 'exit' to end the conversation
Type 'clear' to start a new conversation
--------------------------------------------------

Assistant: To solve for x in the equation 5*x + 3 = 13, we first multiply both sides of the equation by 5 to eliminate the fraction. This gives us 5*x + 3*x = 13*x. 

Next, we expand the equation to get 5*x + 3*x + 3 = 13*x. 

## 12. Deploying in NGROK

In [56]:
%uv pip install pyngrok

Using Python 3.12.6 environment at: /usr/local
Resolved 2 packages in 57ms
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
Prepared 1 package in 9ms
Installed 1 package in 158ms
 + pyngrok==7.5.0
Note: you may need to restart the kernel to use updated packages.


In [65]:
from pyngrok import ngrok
authtoken = "82pQNRYrr19fLeBVzxgnU_7LBDdaEy1z5oEbqw9qKQf"
ngrok.set_auth_token(authtoken)
public_url = ngrok.connect(8000)
print(f'Click to access the Web UI: {public_url}')

Click to access the Web UI: NgrokTunnel: "https://6c0b4c788d7f.ngrok-free.app" -> "http://localhost:8000"


t=2026-01-11T20:47:47+0000 lvl=warn msg="failed to open private leg" id=f7266d9824a8 privaddr=localhost:8000 err="dial tcp 127.0.0.1:8000: connect: connection refused"
t=2026-01-11T20:56:10+0000 lvl=warn msg="failed to open private leg" id=614517422a5a privaddr=localhost:8000 err="dial tcp 127.0.0.1:8000: connect: connection refused"
t=2026-01-11T23:18:38+0000 lvl=warn msg="failed to open private leg" id=8a2ad1aa6ab5 privaddr=localhost:8000 err="dial tcp 127.0.0.1:8000: connect: connection refused"
t=2026-01-11T23:23:04+0000 lvl=warn msg="failed to open private leg" id=60e659b50b3a privaddr=localhost:8000 err="dial tcp 127.0.0.1:8000: connect: connection refused"
t=2026-01-11T23:35:58+0000 lvl=warn msg="Stopping forwarder" name=http-8000-6b56b689-738e-4f96-b807-7b8b524e8667 acceptErr="failed to accept connection: Listener closed"


In [116]:
!bash -c "source .venv/bin/activate && python -m scripts.chat_web --temperature 0.6 --max-tokens 256 --top-k 50"

Autodetected device type: cuda
2026-01-11 23:32:47,984 - nanochat.common - INFO - Distributed world size: 1
Starting NanoChat Web Server
Temperature: 0.6, Top-k: 50, Max tokens: 256
INFO:     Started server process [15312]
INFO:     Waiting for application startup.
Loading nanochat models across GPUs...
Initializing worker pool with 1 GPUs...
Loading model on GPU 0...
2026-01-11 23:32:48,022 - nanochat.checkpoint_manager - INFO - No model tag provided, guessing model tag: d34
2026-01-11 23:32:48,022 - nanochat.checkpoint_manager - INFO - Loading model from /content/nanochat_data/chatsft_checkpoints/d34 with step 669
2026-01-11 23:32:53,705 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 34, 'n_head': 17, 'n_kv_head': 17, 'n_embd': 2176}
All 1 workers initialized!
Server ready at http://localhost:8000
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
IN